In [1]:
!pip install psycopg2-binary pandas sqlalchemy 

import psycopg2
import sqlalchemy
import pandas as pd
import csv
from sqlalchemy import create_engine



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [29]:
# CONEXIÓN
DB_USER = "bancolombia"
DB_PASS = "pruebatecnica"
DB_HOST = "postgresql-bancolombia.alwaysdata.net"
DB_PORT = "5432"
DB_NAME = "bancolombia_prueba"

DATABASE_URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

DB_CONFIG = {
    "host": DB_HOST,
    "user": DB_USER,
    "password": DB_PASS,
    "dbname": DB_NAME
}

def ejecutar_consulta(sql_query):
    try:
        engine = create_engine(DATABASE_URL)
        df = pd.read_sql_query(sql_query, engine)
        return df
    except Exception as e:
        print(f"Error en la consulta: {e}")
        return None
    finally:
        if 'engine' in locals():
            engine.dispose()

def ejecutar_modificacion(sql_command):
    try:
        with psycopg2.connect(**DB_CONFIG) as conexion:
            with conexion.cursor() as cursor:
                cursor.execute(sql_command)
        
        print(f"Comando ejecutado exitosamente.")
        return True
    except (Exception, psycopg2.Error) as error:
        print(f"Ocurrió un error al ejecutar el comando: {error}")
        return False





In [30]:
cambiar_formato_cod_activo_num = """
ALTER TABLE catalogo_activos
ALTER COLUMN cod_activo TYPE INTEGER USING cod_activo::INTEGER;
"""
ejecutar_modificacion(cambiar_formato_cod_activo_num)

cambiar_formato_cod_perfil_num = """
ALTER TABLE cat_perfil_riesgo
ALTER COLUMN cod_perfil_riesgo TYPE INTEGER USING cod_perfil_riesgo::INTEGER;
"""
ejecutar_modificacion(cambiar_formato_cod_perfil_num)

comando_limpieza_fechas = """
    UPDATE historico_aba_macroactivos SET ingestion_year = NULL WHERE ingestion_year !~ '^\d+$';
    UPDATE historico_aba_macroactivos SET ingestion_month = NULL WHERE ingestion_month !~ '^\d+$';
    UPDATE historico_aba_macroactivos SET ingestion_day = NULL WHERE ingestion_day !~ '^\d+$';

    ALTER TABLE historico_aba_macroactivos
        ALTER COLUMN ingestion_year TYPE INTEGER USING ingestion_year::INTEGER,
        ALTER COLUMN ingestion_month TYPE INTEGER USING ingestion_month::INTEGER,
        ALTER COLUMN ingestion_day TYPE INTEGER USING ingestion_day::INTEGER;

    -- Anular meses inválidos
    UPDATE historico_aba_macroactivos
    SET ingestion_month = NULL
    WHERE ingestion_month > 12 OR ingestion_month < 1;

    -- Anular días inválidos
    UPDATE historico_aba_macroactivos
    SET ingestion_day = NULL
    WHERE ingestion_day > 31 OR ingestion_day < 1;

    ALTER TABLE historico_aba_macroactivos ADD COLUMN ingestion_date DATE;

    UPDATE historico_aba_macroactivos
    SET ingestion_date = TO_DATE(ingestion_year || '-' || ingestion_month || '-' || ingestion_day, 'YYYY-MM-DD');

    ALTER TABLE historico_aba_macroactivos
    DROP COLUMN ingestion_year,
    DROP COLUMN ingestion_month,
    DROP COLUMN ingestion_day,
    DROP COLUMN year,
    DROP COLUMN month;
"""
ejecutar_modificacion(comando_limpieza_fechas)
ejecutar_consulta("SELECT * FROM historico_aba_macroactivos LIMIT 100")


<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_3538/3711460865.py:13: SyntaxWarning: invalid escape sequence '\d'
  comando_limpieza_fechas = """


Comando ejecutado exitosamente.
Comando ejecutado exitosamente.
Comando ejecutado exitosamente.


,id_historico,id_sistema_cliente,macroactivo,cod_activo,aba,cod_perfil_riesgo,cod_banca,ingestion_date
0,265,10039616197,,1005,95300,1466,PN,2024-04-11
1,6645,1.009E+11,FICs,10007,957893147.1,1467,PF,2024-02-01
2,6646,10039616197,Renta Variable,1004,2075000,1466,PN,2024-03-13
3,6647,10079142847,Renta Variable,1014,4022700,1468,PF,2024-04-05
4,6648,10038643094,FICs,1018,86445655.81,1466,PN,2024-05-06
...,...,...,...,...,...,...,...,...
95,69,10032184607,Renta Variable,1011,7370600,1468,PN,2023-12-14
96,70,10039616197,Renta Variable,1004,2355000,1466,PN,2024-02-07
97,72,10039538018,FICs,10007,339045.18,1468,,2024-04-29
98,73,1.00114E+12,Renta Variable,1004,2240000,1466,PN,2024-04-15


In [31]:
comando_limpieza_fechas_usd = """
    UPDATE historico_aba_usd_internacional SET ingestion_year = NULL WHERE ingestion_year !~ '^\d+$';
    UPDATE historico_aba_usd_internacional SET ingestion_month = NULL WHERE ingestion_month !~ '^\d+$';
    UPDATE historico_aba_usd_internacional SET ingestion_day = NULL WHERE ingestion_day !~ '^\d+$';

    ALTER TABLE historico_aba_usd_internacional
        ALTER COLUMN ingestion_year TYPE INTEGER USING ingestion_year::INTEGER,
        ALTER COLUMN ingestion_month TYPE INTEGER USING ingestion_month::INTEGER,
        ALTER COLUMN ingestion_day TYPE INTEGER USING ingestion_day::INTEGER;

    -- Anular meses inválidos
    UPDATE historico_aba_usd_internacional
    SET ingestion_month = NULL
    WHERE ingestion_month > 12 OR ingestion_month < 1;

    -- Anular días inválidos
    UPDATE historico_aba_usd_internacional
    SET ingestion_day = NULL
    WHERE ingestion_day > 31 OR ingestion_day < 1;

    ALTER TABLE historico_aba_usd_internacional ADD COLUMN ingestion_date DATE;

    UPDATE historico_aba_usd_internacional
    SET ingestion_date = TO_DATE(ingestion_year || '-' || ingestion_month || '-' || ingestion_day, 'YYYY-MM-DD');

    ALTER TABLE historico_aba_usd_internacional
    DROP COLUMN ingestion_year,
    DROP COLUMN ingestion_month,
    DROP COLUMN ingestion_day;
"""
ejecutar_modificacion(comando_limpieza_fechas_usd)
ejecutar_consulta("SELECT * FROM historico_aba_usd_internacional LIMIT 100")


<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_3538/285905980.py:1: SyntaxWarning: invalid escape sequence '\d'
  comando_limpieza_fechas_usd = """


Comando ejecutado exitosamente.


,id_historico,id_sistema_cliente,simbol,cusip,isin,nombre_activo,cantidad,valor_mercado,fecha_vencimiento,tasa_cupon,ingestion_date
0,61,1004870235,AAPL,37833100,US0378331005,APPLE INC COM,1466,282322.28,1/01/1900,0,2024-03-01
1,323,1004870235,AAPL,37833100,US0378331005,APPLE INC COM,1466,255391.86,1/01/1900,0,2024-03-01
2,529,1004870235,NFLX,64110L106,US64110L1061,NETFLIX INC COM,19,7174.21,1/01/1900,0,2024-03-01
3,1724,10020203023,nan,L1049H415,LU0072462426,BGF GLOBAL ALLOCATION FUND CLASS A2 (USD),712.09,48792.41,1/01/1900,0,2024-03-01
4,1910,10021382511,nan,L5785X484,LU1041599405,JPMORGAN INCOME FUND CLASS A (ACC)(USD),797.037,99302.84,1/01/1900,0,2024-03-01
...,...,...,...,...,...,...,...,...,...,...,...
95,66,1004870235,AAPL,37833100,US0378331005,APPLE INC COM,1466,277616.42,1/01/1900,0,2024-03-01
96,67,1004870235,nan,L1R49P707,LU1196525536,BGF CONTINENTAL EUROPEAN FLEXIBLE FUND CLASS A...,5973.45,135348.45,1/01/1900,0,2024-03-01
97,68,1004870235,nan,L4058R217,LU0109391861,FRANKLIN U.S. OPPORTUNITIES FUND CLASS A (ACC)...,6249.938,177435.74,1/01/1900,0,2024-03-01
98,69,1004870235,VGK,922042874,US9220428745,VANGUARD INTL EQUITY INDEX FDS FTSE EUROPE ETF,286,18549.96,1/01/1900,0,2024-03-01


In [35]:
tabla_a_revisar = 'historico_aba_usd_internacional'

consulta_de_tipos = f"""
    SELECT 
        column_name, 
        data_type 
    FROM 
        information_schema.columns 
    WHERE 
        table_name = '{tabla_a_revisar}';
"""

print(f"Revisando los tipos de dato para la tabla: '{tabla_a_revisar}'...")
df_tipos = ejecutar_consulta(consulta_de_tipos)

if df_tipos is not None:
    print(df_tipos)

Revisando los tipos de dato para la tabla: 'historico_aba_usd_internacional'...
          column_name          data_type
0        id_historico            integer
1            cantidad            numeric
2       valor_mercado            numeric
3      ingestion_date               date
4                isin               text
5       nombre_activo               text
6  id_sistema_cliente  character varying
7              simbol  character varying
8               cusip  character varying


In [33]:
limpieza_tipos_macro = """
UPDATE historico_aba_macroactivos 
SET 
    cod_activo = NULL 
WHERE 
    cod_activo::TEXT !~ '^\d+$'; 

UPDATE historico_aba_macroactivos 
SET 
    cod_perfil_riesgo = NULL 
WHERE 
    cod_perfil_riesgo::TEXT !~ '^\d+$'; 

UPDATE historico_aba_macroactivos 
SET 
    aba = NULL 
WHERE 
    aba::TEXT !~ '^-?\d+(\.\d+)?$'; 

ALTER TABLE historico_aba_macroactivos
    ALTER COLUMN cod_activo TYPE INTEGER USING cod_activo::INTEGER,
    ALTER COLUMN cod_perfil_riesgo TYPE INTEGER USING cod_perfil_riesgo::INTEGER,
    ALTER COLUMN aba TYPE NUMERIC(25, 4) USING aba::NUMERIC;

"""
ejecutar_modificacion(limpieza_tipos_macro)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_3538/3817910965.py:1: SyntaxWarning: invalid escape sequence '\d'
  limpieza_tipos_macro = """


Comando ejecutado exitosamente.


True

In [34]:
limpieza_tipos_usd = """
UPDATE historico_aba_usd_internacional
SET 
    cantidad = NULL 
WHERE 
    cantidad::TEXT !~ '^-?\d+(\.\d+)?$'; 

UPDATE historico_aba_usd_internacional
SET 
    valor_mercado = NULL 
WHERE 
    valor_mercado::TEXT !~ '^-?\d+(\.\d+)?$'; 
 

ALTER TABLE historico_aba_usd_internacional
    ALTER COLUMN cantidad TYPE NUMERIC(25, 4) USING cantidad::NUMERIC,
    ALTER COLUMN valor_mercado TYPE NUMERIC(25, 4) USING valor_mercado::NUMERIC,
    DROP COLUMN fecha_vencimiento,
    DROP COLUMN tasa_cupon;
"""

ejecutar_modificacion(limpieza_tipos_usd)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_3538/465518894.py:1: SyntaxWarning: invalid escape sequence '\d'
  limpieza_tipos_usd = """


Comando ejecutado exitosamente.


True

In [36]:
# DEFINIR LAS CLAVES FORANEAS

claves_foraneas = """
UPDATE historico_aba_macroactivos
SET cod_activo = NULL
WHERE cod_activo IS NOT NULL AND cod_activo NOT IN (SELECT cod_activo FROM catalogo_activos);

UPDATE historico_aba_macroactivos
SET cod_perfil_riesgo = NULL
WHERE cod_perfil_riesgo IS NOT NULL AND cod_perfil_riesgo NOT IN (SELECT cod_perfil_riesgo FROM cat_perfil_riesgo);

UPDATE historico_aba_macroactivos
SET cod_banca = NULL
WHERE cod_banca IS NOT NULL AND cod_banca NOT IN (SELECT cod_banca FROM catalogo_banca);

ALTER TABLE historico_aba_macroactivos
    ADD CONSTRAINT fk_activo FOREIGN KEY (cod_activo) REFERENCES catalogo_activos(cod_activo),
    ADD CONSTRAINT fk_perfil_riesgo FOREIGN KEY (cod_perfil_riesgo) REFERENCES cat_perfil_riesgo(cod_perfil_riesgo),
    ADD CONSTRAINT fk_banca FOREIGN KEY (cod_banca) REFERENCES catalogo_banca(cod_banca);
"""

ejecutar_modificacion(claves_foraneas)


Comando ejecutado exitosamente.


True

In [37]:
consulta_de_prueba = """
SELECT 
    pr.perfil_riesgo, 
    AVG(h.aba) as promedio_aba
FROM 
    historico_aba_macroactivos h
JOIN 
    cat_perfil_riesgo pr ON h.cod_perfil_riesgo = pr.cod_perfil_riesgo
WHERE
    h.aba IS NOT NULL 
GROUP BY 
    pr.perfil_riesgo
ORDER BY 
    promedio_aba DESC;
"""

ejecutar_consulta(consulta_de_prueba)

,perfil_riesgo,promedio_aba
0,CONSERVADOR,5.212776e+08
1,MODERADO,8.081549e+07
2,SIN DEFINIR,6.961319e+06
3,AGRESIVO,4.068154e+06
